In [1]:
# multivariate multi-step 1d cnn example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  
from sklearn.metrics import r2_score # R square

In [2]:
excelFile = 'F:\\wang\\自学\\数据\\Preprocessed Data.xlsx'
#读取数据，指定日期列为指标，Pandas自动将“日期”列识别为Datetime格式
data = pd.read_excel(excelFile,sheet_name=0)
data['Date '] = data['Date '].apply(lambda x:x.strftime('%Y-%m-%d'))
data['Time (hh:mm)'] = data['Time (hh:mm)'].apply(lambda x:x.strftime('%H:%M:%S'))
data['datetime'] =pd.to_datetime(data['Date '] + ' ' + data['Time (hh:mm)'])
data.drop(list(data)[0:2],axis=1,inplace=True)
# data=data.set_index('datetime')
data.head()

,Ambient temperature (°C),Global plane of array irradiance (W/m²),m-Si module temperature (°C),DC power of m-Si (W),datetime
0,18.771,205.614,21.213,348,2018-05-25 07:10:01
1,17.960,266.586,20.795,412,2018-05-25 07:25:01
2,18.432,278.943,23.032,569,2018-05-25 07:45:01
3,18.397,210.042,23.655,426,2018-05-25 07:50:01
4,18.178,249.752,22.766,489,2018-05-25 08:00:01


In [3]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [40]:
# define input sequence
in_seq1 = data['Ambient temperature (°C)'].values
in_seq2 = data['Global plane of array irradiance (W/m²)'].values
in_seq3 = data['m-Si module temperature (°C)'].values
in_seq4 = data['DC power of m-Si (W)'].values
out_seq = data['DC power of m-Si (W)'].values
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
in_seq3 = in_seq3.reshape((len(in_seq3), 1))
in_seq4 = in_seq4.reshape((len(in_seq4), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2,in_seq3,in_seq4, out_seq))
# choose a number of time steps
n_steps_in, n_steps_out =117 , 3
# convert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
# flatten input
n_input = X.shape[1] * X.shape[2]
n_features = X.shape[2]
# X = X.reshape((X.shape[0], n_input))
X_train = X[:int(len(X)*0.7)]
y_train = y[:int(len(X)*0.7)]
X_test = X[int(len(X)*0.7):]
y_test = y[int(len(X)*0.7):]
# define model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps_in, n_features)))
# model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X_train, y_train, epochs=350, verbose=1)
# demonstrate prediction
# x_input = array([[70, 75], [80, 85], [90, 95]])
# x_input = x_input.reshape((1, n_input))
# yhat = model.predict(x_input, verbose=0)
# print(yhat)

Epoch 1/350
110/110 [==============================] - 2s 9ms/step - loss: 234952.2162
Epoch 2/350
110/110 [==============================] - 1s 10ms/step - loss: 29096.9360
Epoch 3/350
110/110 [==============================] - 1s 10ms/step - loss: 18269.6133
Epoch 4/350
110/110 [==============================] - 1s 9ms/step - loss: 28175.9266
Epoch 5/350
110/110 [==============================] - 1s 9ms/step - loss: 14286.5954
Epoch 6/350
110/110 [==============================] - 1s 9ms/step - loss: 16368.0120
Epoch 7/350
110/110 [==============================] - 1s 8ms/step - loss: 11287.5465
Epoch 8/350
110/110 [==============================] - 1s 9ms/step - loss: 13242.6416
Epoch 9/350
110/110 [==============================] - 1s 9ms/step - loss: 12569.7212
Epoch 10/350
110/110 [==============================] - 1s 9ms/step - loss: 13789.7823
Epoch 11/350
110/110 [==============================] - 1s 9ms/step - loss: 11165.0398
Epoch 12/350
110/110 [===========================

110/110 [==============================] - 1s 9ms/step - loss: 3512.5429
Epoch 97/350
110/110 [==============================] - 1s 9ms/step - loss: 2228.5121
Epoch 98/350
110/110 [==============================] - 1s 9ms/step - loss: 2437.7287
Epoch 99/350
110/110 [==============================] - 1s 9ms/step - loss: 1977.2590
Epoch 100/350
110/110 [==============================] - 1s 9ms/step - loss: 2871.4750
Epoch 101/350
110/110 [==============================] - 1s 9ms/step - loss: 2321.7845
Epoch 102/350
110/110 [==============================] - 1s 9ms/step - loss: 2313.5239
Epoch 103/350
110/110 [==============================] - 1s 9ms/step - loss: 2297.1272
Epoch 104/350
110/110 [==============================] - 1s 9ms/step - loss: 2357.1869
Epoch 105/350
110/110 [==============================] - 1s 9ms/step - loss: 2124.8154
Epoch 106/350
110/110 [==============================] - 1s 9ms/step - loss: 2734.2606
Epoch 107/350
110/110 [==============================] - 1s 

110/110 [==============================] - 1s 10ms/step - loss: 1044.0622
Epoch 191/350
110/110 [==============================] - 1s 10ms/step - loss: 636.7284: 0s 
Epoch 192/350
110/110 [==============================] - 1s 9ms/step - loss: 1391.3217
Epoch 193/350
110/110 [==============================] - 1s 9ms/step - loss: 1204.0446
Epoch 194/350
110/110 [==============================] - 1s 9ms/step - loss: 959.0292
Epoch 195/350
110/110 [==============================] - 1s 9ms/step - loss: 1429.5104
Epoch 196/350
110/110 [==============================] - 1s 9ms/step - loss: 767.0174
Epoch 197/350
110/110 [==============================] - 1s 9ms/step - loss: 722.6716
Epoch 198/350
110/110 [==============================] - 1s 9ms/step - loss: 736.2152
Epoch 199/350
110/110 [==============================] - 1s 9ms/step - loss: 615.6632
Epoch 200/350
110/110 [==============================] - 1s 9ms/step - loss: 843.8422
Epoch 201/350
110/110 [==============================] - 

110/110 [==============================] - 1s 9ms/step - loss: 2475.0323
Epoch 286/350
110/110 [==============================] - 1s 9ms/step - loss: 959.0243
Epoch 287/350
110/110 [==============================] - 1s 9ms/step - loss: 502.3832
Epoch 288/350
110/110 [==============================] - 1s 9ms/step - loss: 343.0283
Epoch 289/350
110/110 [==============================] - 1s 9ms/step - loss: 359.8790
Epoch 290/350
110/110 [==============================] - 1s 9ms/step - loss: 346.3153
Epoch 291/350
110/110 [==============================] - 1s 9ms/step - loss: 435.4944
Epoch 292/350
110/110 [==============================] - 1s 9ms/step - loss: 359.9592
Epoch 293/350
110/110 [==============================] - 1s 9ms/step - loss: 336.6144
Epoch 294/350
110/110 [==============================] - 1s 9ms/step - loss: 339.8583
Epoch 295/350
110/110 [==============================] - 1s 9ms/step - loss: 327.8738
Epoch 296/350
110/110 [==============================] - 1s 9ms/ste

In [41]:
yhat = model.predict(X_test, verbose=0)
yhat

array([[1767.1678 , 1745.8036 , 1730.5574 ],
       [1765.2599 , 1746.399  , 1715.393  ],
       [1746.6337 , 1719.7883 , 1694.5754 ],
       ...,
       [ 410.09604,  366.40857,  314.95407],
       [ 342.98956,  341.64468,  266.37192],
       [ 321.1411 ,  262.72015,  242.52455]], dtype=float32)

In [42]:
y_test

array([[1736., 1708., 1702.],
       [1708., 1702., 1688.],
       [1702., 1688., 1672.],
       ...,
       [ 384.,  341.,  310.],
       [ 341.,  310.,  274.],
       [ 310.,  274.,  241.]])

In [43]:
y_t = pd.DataFrame(y_test)
y_p = pd.DataFrame(yhat)

In [44]:
def evaluate(y_true,y_pred):
    SSE = sum((y_true-y_pred)**2)
    MSE = SSE/len(y_true)
    RMSE = MSE**(1/2)
    MAE = sum(abs(y_true-y_pred))/len(y_true)
    MAPE = sum(abs(y_true-y_pred)/y_true)/len(y_true)*100
    SD = (sum((abs(y_true-y_pred)-sum(abs(y_true-y_pred))/len(y_true))**2)/len(y_true))**(1/2)
    r2s = r2_score(y_true, y_pred)
    print('MSE: ', MSE, '\n', 'RMSE: ', RMSE, '\n','MAE: ', MAE, '\n','MAPE: ', MAPE, '\n','SD: ', SD, '\n', 'R square: ', r2s)

In [48]:
evaluate(y_t[0],y_p[0])

MSE:  1367.0298705566486 
 RMSE:  36.973367043814775 
 MAE:  33.74812028242305 
 MAPE:  2.803283843065757 
 SD:  15.102789409898907 
 R square:  0.9936161056677697
